In [1]:
from numpy.random import random as randomArray
import numpy as np
import pyopencl as cl

%load_ext pyopencl.ipython_ext

In [2]:
def opencl(data):
    #array = np.ma.array(randomArray((8*256,8*256))*255, np.double)
    #someArray = np.array(randomArray((256,256))*255, np.float32)

    #someArray

    # Context
    ctx = cl.create_some_context()
    # Create queue
    queue = cl.CommandQueue(ctx)

    # Get mem flags
    mf = cl.mem_flags

    # Create a read-only buffer on device and copy 'someArray' from host to device
    someArrayAsOpenCLType = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=data)

    # Create a write-only buffer to get the result from device
    resultAsOpenCLType = cl.Buffer(ctx, mf.WRITE_ONLY, data.nbytes)

    program = cl.Program(ctx, """
    __kernel void multiplyByTwo(__global const float4 *someArrayAsOpenCLType, __global float4 *resultAsOpenCLType) {
            int gid = get_global_id(0);

            float4 data = someArrayAsOpenCLType[gid];
            resultAsOpenCLType[gid] =  data * (float) 2.0;
    }
    """).build()

    program.multiplyByTwo(queue, data.shape, None, someArrayAsOpenCLType, resultAsOpenCLType)

    # Creates a buffer for the result (host memory)
    result = np.empty_like(data)

    # Copy the results from device to host
    cl.enqueue_copy(queue, result, resultAsOpenCLType)

In [16]:
array = np.array(randomArray((10*2048,10*2048))*255, np.float32)

In [21]:
# with opencl 
%timeit opencl(array)

1 loop, best of 3: 1.78 s per loop


In [20]:
# no opencl
%timeit array * 2.0

1 loop, best of 3: 462 ms per loop
